In [1]:
import cloudscraper as cs
import bs4
import json
import datetime
import pickle
import requests
import os
import numpy

# Classes

In [67]:
class Collection:
    def __init__(self, data):
        self.slug = data["slug"]
        self.name = data["name"]
        self.description = data["description"]
        self.bannerImageUrl = data["bannerImageUrl"]
        self.imageUrl = data["imageUrl"]
        self.relayId = data["relayId"]
        self.isVerified = data["isVerified"]
        self.isMintable = data["isMintable"]
        self.isSafelisted = data["isSafelisted"]
        self.owner = data["owner"]
        self.stats = data["stats"]
        self.nativePaymentAsset = data["nativePaymentAsset"]
        self.statsV2 = data["statsV2"]
        self.isWatching = data["isWatching"]
        self.id = data["id"]
        self.assetCount = data["assetCount"]
        self.includeTradingHistory = data["includeTradingHistory"]
        self.numericTraits = data["numericTraits"]
        self.stringTraits = data["stringTraits"]
        self.paymentAssets = data["paymentAssets"]

### Helpers

In [2]:
def parse_asset_data(asset_dict):
    
    asset_id = asset_dict['token_id']
    
    try:
        creator_username = asset_dict['creator']['user']['username']
    except:
        creator_username = None
    try:
        creator_address = asset_dict['creator']['address']
    except:
        creator_address = None
    
    try:
        owner_username = asset_dict['owner']['user']['username']
    except:
        owner_username = None
    
    owner_address = asset_dict['owner']['address']
    
    traits = asset_dict['traits']
    num_sales = int(asset_dict['num_sales'])
        
    result = {'asset_id': asset_id,
              'creator_username': creator_username,
              'creator_address': creator_address,
              'owner_username': owner_username,
              'owner_address': owner_address,
              'traits': traits,
              'num_sales': num_sales}
    
    return result


def parse_sale_data(sale_dict):
    
    is_bundle = False

    if sale_dict['asset'] != None:
        asset_id = sale_dict['asset']['token_id']
    elif sale_dict['asset_bundle'] != None:
        asset_id = [asset['token_id'] for asset in sale_dict['asset_bundle']['assets']]
        is_bundle = True
    
    
    seller_address = sale_dict['seller']['address']
    buyer_address = sale_dict['winner_account']['address']
    
    try:
        seller_username = sale_dict['seller']['user']['username']
    except:
        seller_username = None    
    try:
        buyer_username = sale_dict['winner_account']['user']['username']
    except:
        buyer_username = None
    
    timestamp = sale_dict['transaction']['timestamp']
    total_price = float(sale_dict['total_price'])
    payment_token = sale_dict['payment_token']['symbol']
    usd_price = float(sale_dict['payment_token']['usd_price'])
    transaction_hash = sale_dict['transaction']['transaction_hash']
    

    result = {'is_bundle': is_bundle,
              'asset_id': asset_id,
              'seller_address': seller_address,
              'buyer_address': buyer_address,
              'buyer_username': buyer_username,
              'seller_username':seller_username,
              'timestamp': timestamp,
              'total_price': total_price, 
              'payment_token': payment_token,
              'usd_price': usd_price,
              'transaction_hash': transaction_hash}
    
    return result

In [3]:
import csv

def writeCSV(fname, data):
    with open(fname, 'w', newline='') as file:
        writer = csv.writer(file)
        
        for el in data:
            writer.writerow(el)

In [4]:
with open(f"{os.getcwd()}/opensea-api-key") as f:
    OS_API_KEY = f.read()[:-2]

# Get Slugs

In [5]:
files = [f for f in os.listdir() if f[-4:] == ".pkl"] 
print(files)

data = {}
for file in files:
    with open(file, 'rb') as f:
        loaded_file = pickle.load(f)
        data[file] = loaded_file

['25-02-2022_11-10-56_rankings.pkl']


In [6]:
d = data[files[0]]

In [7]:
d

{'3landers': {'__id': 'Q29sbGVjdGlvblR5cGU6NzY1MTU3MA==',
  '__typename': 'CollectionType',
  'createdDate': '2022-02-19T13:01:18.111966',
  'name': '3Landers',
  'slug': '3landers',
  'logo': 'https://lh3.googleusercontent.com/jM8itRiSM3hI8RWokMkhR97JdoIzukU3DUEO-9MD3i6r6DnTBW6efbv1617zES0MfoiWzW4f3UaL9lgRr61vf3Cj4DDPIsk_XjNBXQ=s120',
  'isVerified': False,
  'nativePaymentAsset': {'__ref': 'UGF5bWVudEFzc2V0VHlwZTo0Mg=='},
  'statsV2': {'__ref': 'client:Q29sbGVjdGlvblR5cGU6NzY1MTU3MA==:statsV2'},
  'id': 'Q29sbGVjdGlvblR5cGU6NzY1MTU3MA=='},
 'tubby-cats': {'__id': 'Q29sbGVjdGlvblR5cGU6Nzc1MDIzNQ==',
  '__typename': 'CollectionType',
  'createdDate': '2022-02-21T16:56:00.741611',
  'name': 'tubby cats by tubby collective',
  'slug': 'tubby-cats',
  'logo': 'https://lh3.googleusercontent.com/XHZY9623keDQqFSDHKqOdcjD99Y7N82K1egYRM2Mm1Z-Jxn5myrkKiC5NBktWKStVtTzDzwELy9dNpzTWJTIkLsdMIxUHI86jduQ=s120',
  'isVerified': False,
  'nativePaymentAsset': {'__ref': 'UGF5bWVudEFzc2V0VHlwZTo0Mg=='},


# Cloudscraper

In [21]:
scraper = cs.CloudScraper()
offset = len("window.__wired__=")

# Pull collections

In [69]:
url = "https://opensea.io/collection/"
query = "?search[sortAscending]=true&search[sortBy]=PRICE"

soup = bs4.BeautifulSoup(scraper.get(url).text, "html.parser")

def pullCollection(slug):
    data = []
    
    soup = bs4.BeautifulSoup(scraper.get(url + slug + query).text, "html.parser")
    script = soup.script.contents[0][offset:]

    decoded_script = json.loads(script)["records"]
        
    return decoded_script

def parseCollection(slug):
    coll = pullCollection(slug)
    asset_data = {}

    for k in coll.keys():
        if coll[k]["__typename"] == "CollectionType" and coll[k]["slug"] == slug:
            coll_data = coll[k]
        elif coll[k]["__typename"] == "AssetType":
            asset_data[k] = coll[k]
    
    return coll_data, asset_data

In [70]:
a, b = parseCollection("3landers")
print(len(a))
print(len(b))

33
35


In [79]:
c = Collection(a)

In [80]:
print(a)

{'__id': 'Q29sbGVjdGlvblR5cGU6NzY1MTU3MA==', '__typename': 'CollectionType', 'isEditable': False, 'bannerImageUrl': 'https://lh3.googleusercontent.com/nzpEHnEfOg65RF6LO9sGm-hbLa7rfR0Nu70rFYp9--AMKNbVckXs4QohYrUDABZefZkZ01JeZH7ANSvea8xKfMjRxgbyl8MQOfDw=s2500', 'name': '3Landers', 'description': '3Landers is a collectible NFT project centered around community, adventure, and collaboration. Each 3Lander resides on the Ethereum blockchain as a unique, non-fungible token (NFT) made up out of a unique combination of traits and underlying “DNAs”. Holding a 3Landers NFT makes you a 3Lander – a member of a world and community that serves to build meaningful long-term connection through collaboration, adventure, building, creating, and dreaming!', 'imageUrl': 'https://lh3.googleusercontent.com/jM8itRiSM3hI8RWokMkhR97JdoIzukU3DUEO-9MD3i6r6DnTBW6efbv1617zES0MfoiWzW4f3UaL9lgRr61vf3Cj4DDPIsk_XjNBXQ=s120', 'relayId': 'Q29sbGVjdGlvblR5cGU6NzY1MTU3MA==', 'connectedTwitterUsername': '3landersNFT', 'repr

In [75]:
for el in b.values():
    temp = el
    print(el)
    print()

{'__id': 'QXNzZXRUeXBlOjMwMDQ2MTM4Mg==', '__typename': 'AssetType', 'assetContract': {'__ref': 'QXNzZXRDb250cmFjdFR5cGU6NTQyNDA0'}, 'id': 'QXNzZXRUeXBlOjMwMDQ2MTM4Mg=='}

{'__id': 'QXNzZXRUeXBlOjEzNjg5MDc3', '__typename': 'AssetType', 'imageUrl': 'https://storage.opensea.io/files/6f8e2979d428180222796ff4a33ab929.svg', 'id': 'QXNzZXRUeXBlOjEzNjg5MDc3', 'decimals': 18, 'symbol': 'ETH', 'usdSpotPrice': 2618.9, 'assetContract': {'__ref': 'QXNzZXRDb250cmFjdFR5cGU6MjMzMQ=='}}

{'__id': 'QXNzZXRUeXBlOjMwMDMwNjE0Mg==', '__typename': 'AssetType', 'assetContract': {'__ref': 'QXNzZXRDb250cmFjdFR5cGU6NTQyNDA0'}, 'collection': {'__ref': 'Q29sbGVjdGlvblR5cGU6NzY1MTU3MA=='}, 'relayId': 'QXNzZXRUeXBlOjMwMDMwNjE0Mg==', 'tokenId': '3250', 'backgroundColor': None, 'imageUrl': 'https://lh3.googleusercontent.com/4gJ_0zXvNZkT3G1Ms4OxEHS8FOrpETT7yFIi9oxHLv9xMUHCi7Cst77nNtag0aT7sOZ7-dVPAMg2e1Y5mH20nwKYpSQuKMwvqTCM', 'name': None, 'id': 'QXNzZXRUeXBlOjMwMDMwNjE0Mg==', 'isDelisted': False, 'animationUrl': None,

In [ ]:
class Asset:
    def __init__(self, data):
        self.ass

In [81]:
temp

{'__id': 'QXNzZXRUeXBlOjMwMDM0MzA1Mw==',
 '__typename': 'AssetType',
 'assetContract': {'__ref': 'QXNzZXRDb250cmFjdFR5cGU6NTQyNDA0'},
 'collection': {'__ref': 'Q29sbGVjdGlvblR5cGU6NzY1MTU3MA=='},
 'relayId': 'QXNzZXRUeXBlOjMwMDM0MzA1Mw==',
 'tokenId': '3538',
 'backgroundColor': None,
 'imageUrl': 'https://lh3.googleusercontent.com/zT6bSdcIN6qjJM4e3TIexFPc0fvDXtIGShOZoj8F0ldsy61ELYE4R2dsJ10u37tW6KunySlVurHnH637d09w8y2d69ribLK9qf-R6Q',
 'name': None,
 'id': 'QXNzZXRUeXBlOjMwMDM0MzA1Mw==',
 'isDelisted': False,
 'animationUrl': None,
 'displayImageUrl': 'https://lh3.googleusercontent.com/zT6bSdcIN6qjJM4e3TIexFPc0fvDXtIGShOZoj8F0ldsy61ELYE4R2dsJ10u37tW6KunySlVurHnH637d09w8y2d69ribLK9qf-R6Q',
 'decimals': 0,
 'favoritesCount': 1,
 'isFavorite': False,
 'isFrozen': False,
 'hasUnlockableContent': False,
 'orderData': {'__ref': 'client:QXNzZXRUeXBlOjMwMDM0MzA1Mw==:orderData'},
 'assetEventData': {'__ref': 'client:QXNzZXRUeXBlOjMwMDM0MzA1Mw==:assetEventData'},
 'isEditable': {'__ref': 'client